### Урок 2. #Профилирование пользователей. Сегментация аудитории: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)#

### Домашнее задание

1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [1]:
!pip install gensim --user

In [2]:
!pip install razdel

In [3]:
!pip install --upgrade razdel

Requirement already up-to-date: razdel in c:\programdata\anaconda3\lib\site-packages (0.5.0)


In [11]:
!pip install nltk --user

In [4]:
!pip install --upgrade nltk --user

  Using cached nltk-3.6.5-py3-none-any.whl (1.5 MB)
  Using cached regex-2021.11.10-cp38-cp38-win_amd64.whl (273 kB)


In [5]:
!pip install pymorphy2

In [6]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [21]:
from gensim.models import LdaModel

Наши новости

In [7]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [8]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### Получаем векторные представления новостей

In [9]:
#from gensim.test.utils import common_texts

from gensim.corpora.dictionary import Dictionary

In [14]:
#предобработка текстов
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel

import pymorphy2

In [15]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

In [16]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [17]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [18]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-17-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 24 s


In [19]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 3min 18s


А теперь в 3 строчки обучим нашу модель

In [20]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

Запускаем обучение

In [22]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 33.5 s


In [23]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [24]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(4, 0.04012583), (7, 0.49354932), (8, 0.41572872), (18, 0.031865135)]

In [25]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: северный исследование расчёт американский поток это способность
topic_1: произойти погибнуть тело результат человек район граница
topic_2: обнаружить выяснить мозг год жизнь врач возраст
topic_3: путин президент владимир египет россия российский пресссекретарить
topic_4: продукция сон сотрудничать производить комплексный столетие сербия
topic_5: год британский сообщать который превысить летний nn
topic_6: космос налог лодка сценарий вдвое армения мотив
topic_7: это год который мочь свой весь всё
topic_8: смерть век земля восток планета рейс nn
topic_9: экономика технология украина экономический москва источник медведев
topic_10: женщина мужчина дело суд убийство сотрудник площадь
topic_11: млрд ракета который исследование год это компания
topic_12: сша пенсия дональд доля обращение долг миллиард
topic_13: газ взрыв остров лётчик земля выдавать рот
topic_14: статья рак журнал ст разместить земля полоса
topic_15: фонд век римский тверской молдавия шотландия калининградский
topic

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [26]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [27]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.000000,0.0,0.0,0.000000,0.071642,0.0,0.000000,0.000000,...,0.0,0.115754,0.0,0.0,0.0,0.0,0.0,0.26042,0.0,0.0
1,4896,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.521232,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
2,4897,0.0,0.058491,0.0,0.0,0.039163,0.000000,0.0,0.477272,0.406342,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
3,4898,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.813827,0.175300,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
4,4899,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.184550,...,0.0,0.229215,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0


### Следующий шаг - векторные представления пользователей

In [28]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [29]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [30]:
doc_dict[293622]

array([0.        , 0.        , 0.1380816 , 0.07865322, 0.        ,
       0.20600556, 0.05920433, 0.        , 0.        , 0.02079003,
       0.        , 0.19035743, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.1682716 , 0.        , 0.        ,
       0.        , 0.01780012, 0.11112883, 0.        , 0.        ])

In [34]:
def get_user_embedding(user_articles_list, method='mean'):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    if method == 'mean':
        user_vector = np.mean(user_vector, axis=0)  # Вот тут находим среднее по темам для юзера
    elif method == 'median':
        user_vector = np.median(user_vector, axis=0)
    elif method == 'max':
        user_vector = np.max(user_vector, 0)  # идентично axis=0
    return user_vector

In [35]:
user_articles_list = users['articles'].iloc[33]
get_user_embedding(user_articles_list, method='mean')

array([0.        , 0.        , 0.0065105 , 0.12596269, 0.        ,
       0.        , 0.        , 0.31414554, 0.00664002, 0.00464682,
       0.0771642 , 0.04205361, 0.02379352, 0.0174577 , 0.00205933,
       0.        , 0.03046059, 0.11571998, 0.        , 0.02914587,
       0.01291129, 0.        , 0.13511749, 0.        , 0.04489334])

In [36]:
user_embeddings_mean = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, method='mean'), 1)])
user_embeddings_mean.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_mean['uid'] = users['uid'].values
user_embeddings_mean = user_embeddings_mean[['uid']+['topic_{}'.format(i) for i in range(25)]]

In [37]:
user_embeddings_median = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, method='median'), 1)])
user_embeddings_median.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_median['uid'] = users['uid'].values
user_embeddings_median = user_embeddings_median[['uid']+['topic_{}'.format(i) for i in range(25)]]

In [38]:
user_embeddings_max = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, method='max'), 1)])
user_embeddings_max.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_max['uid'] = users['uid'].values
user_embeddings_max = user_embeddings_max[['uid']+['topic_{}'.format(i) for i in range(25)]]

Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [39]:
target = pd.read_csv("users_churn.csv")

In [56]:
def make_predictions(user_embeddings, target):
    X = pd.merge(user_embeddings, target, 'left')
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)
    logreg = LogisticRegression()
    #обучим
    logreg.fit(X_train, y_train)
    #наши прогнозы для тестовой выборки
    preds = logreg.predict_proba(X_test)[:, 1]
    return preds, y_test

In [57]:
preds_embeddings_mean, y_test_mean = make_predictions(user_embeddings_mean, target)
preds_embeddings_median, y_test_median = make_predictions(user_embeddings_median, target)
preds_embeddings_max, y_test_max = make_predictions(user_embeddings_max, target)

### Рассчитаем Precision, Recall, F_score

In [55]:
def metrics_calculation(y_test, preds):

    roc_auc = roc_auc_score(y_test, preds)
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    return thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc

In [59]:
metrics = []
best_threshold, fscore, precision, recall, roc_auc = metrics_calculation(y_test_mean, preds_embeddings_mean)
metrics.append([fscore, precision, recall, roc_auc])

In [60]:
best_threshold, fscore, precision, recall, roc_auc = metrics_calculation(y_test_median, preds_embeddings_median)
metrics.append([fscore, precision, recall, roc_auc])

In [61]:
best_threshold, fscore, precision, recall, roc_auc = metrics_calculation(y_test_max, preds_embeddings_max)
metrics.append([fscore, precision, recall, roc_auc])

In [63]:
metrics_comparison_df = pd.DataFrame(np.array(metrics),
                   columns=['fscore', 'precision', 'recall', 'roc_auc'],
                   index=['preds_embeddings_mean', 'preds_embeddings_median', 'preds_embeddings_max'])
metrics_comparison_df

,fscore,precision,recall,roc_auc
preds_embeddings_mean,0.712838,0.608069,0.861224,0.958565
preds_embeddings_median,0.782101,0.747212,0.820408,0.977166
preds_embeddings_max,0.771800,0.707483,0.848980,0.971980
